In [1]:
#Importing Packages
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score
import pandas as pd
import numpy as np

In [2]:
#Train & Test
train = pd.read_csv('r52-train-all-terms.txt', header=None, sep='\t')
test = pd.read_csv('r52-test-all-terms.txt', header=None, sep='\t')
#Name the features
train.columns = ['label', 'content']
test.columns = ['label', 'content']


In [3]:
xg_reg=xgb.XGBRegressor(objective='reg:linear',colsample_bytree=0.3, learning_rate=0.1,max_depth=5, alpha=10, n_estimators=10)

In [4]:
class GloveVectorizer:
  def __init__(self):
    # load in pre-trained word vectors
    print('Loading word vectors...')
    word2vec = {}
    embedding = []
    idx2word = []
    with open('glove.6B.50d.txt',encoding='utf-8', errors='ignore') as f:
      # is just a space-separated text file in the format:
      # word vec[0] vec[1] vec[2] ...
      for line in f:
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec
        embedding.append(vec)
        idx2word.append(word)
    print('Found %s word vectors.' % len(word2vec))

    # save for later
    self.word2vec = word2vec
    self.embedding = np.array(embedding)
    self.word2idx = {v:k for k,v in enumerate(idx2word)}
    self.V, self.D = self.embedding.shape

  def fit(self, data):
    pass

  def transform(self, data):
    X = np.zeros((len(data), self.D))
    n = 0
    emptycount = 0
    for sentence in data:
      tokens = sentence.lower().split()
      vecs = []
      for word in tokens:
        if word in self.word2vec:
          vec = self.word2vec[word]
          vecs.append(vec)
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = vecs.mean(axis=0)
      else:
        emptycount += 1
      n += 1
    print("Numer of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X

  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)



In [5]:
trainLabelLst = list(dict.fromkeys(train.label))
trainLabelLst

['cocoa',
 'earn',
 'acq',
 'copper',
 'housing',
 'money-supply',
 'coffee',
 'sugar',
 'trade',
 'reserves',
 'ship',
 'cotton',
 'grain',
 'crude',
 'nat-gas',
 'cpi',
 'interest',
 'money-fx',
 'alum',
 'tin',
 'gold',
 'strategic-metal',
 'retail',
 'ipi',
 'iron-steel',
 'rubber',
 'heat',
 'jobs',
 'lei',
 'bop',
 'gnp',
 'zinc',
 'veg-oil',
 'orange',
 'carcass',
 'pet-chem',
 'gas',
 'wpi',
 'livestock',
 'lumber',
 'instal-debt',
 'meal-feed',
 'lead',
 'potato',
 'nickel',
 'cpu',
 'fuel',
 'jet',
 'income',
 'platinum',
 'dlr',
 'tea']

In [6]:
def get_label(label_str_lst, label_lst):
    return_lst = []
    for s1 in label_str_lst:
        for i, label in enumerate(label_lst):
            if s1 == label:
                return_lst.append(i)
    return return_lst

In [7]:
vectorizer = GloveVectorizer()
Xtrain = vectorizer.fit_transform(train.content)
print(len(train['label']))
print(len(get_label(train['label'],trainLabelLst)))
train['label_idx'] = get_label(train['label'],trainLabelLst)
Ytrain = train.label_idx
data_dmmatrix= xgb.DMatrix(data=Xtrain,label=Ytrain)

Loading word vectors...
Found 400000 word vectors.
Numer of samples with no words found: 0 / 6532
6532
6532


C:\Users\franc\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\franc\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [8]:
param = {
    'max_depth': 10,
    'eta': 0.3,
    'objective': 'multi:softmax',
    'num_class': 52} 
epochs = 1000

model = xgb.train(param, data_dmmatrix, epochs)


In [9]:
Xtest = vectorizer.transform(test.content)
test['label_idx'] = get_label(test['label'],trainLabelLst)
print(test.head())
Ytest = test.label_idx

xgb_test = xgb.DMatrix(Xtest, label=Ytest)
predictions = model.predict(xgb_test)
print(predictions[0:5])

Numer of samples with no words found: 0 / 2568
   label                                            content  label_idx
0  trade  asian exporters fear damage from u s japan rif...          8
1  grain  china daily says vermin eat pct grain stocks a...         12
2   ship  australian foreign ship ban ends but nsw ports...         10
3   gold  western mining to open new gold mine in austra...         20
4    acq  sumitomo bank aims at quick recovery from merg...          2
[ 8. 12. 10. 20.  2.]


In [10]:
accuracy_score(Ytest, predictions)

0.8454049844236761